# Longtype
* 차량 쌍(TV, LV) 기준으로 정리하기
* 왜냐하면, 각 쌍별로 SSM이 계산되기 때문임

# Import

In [12]:
import pandas as pd
import numpy as np

from tqdm import tqdm
import warnings
import os

from scipy import stats # Z-score를 이용한 이상값 제거

import math # arctangent; math.atan 사용 목적
import statistics

# Load Dataset

In [13]:
working_dir = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/011_SSM실증분석_230315-/Dataset/'

In [14]:
data_folder = '02_processed'
data_name = 'merging_type1_03frm_0.10sec.csv'
data_path = os.path.join(working_dir, data_folder, data_name)

In [15]:
df = pd.read_csv(data_path, encoding = 'cp949')

In [16]:
df.head(3)

,Unnamed: 0,Vehicle ID,frame,Total Frames,Global Time (Epoch Time),local_x,local_y,V_len,V_wid,Vehicle Class,...,LVL_rel_velocity,LVL_rel_acc_x,LVL_rel_acc_y,LVL_rel_acc,LVR_rel_velocity_x,LVR_rel_velocity_y,LVR_rel_velocity,LVR_rel_acc_x,LVR_rel_acc_y,LVR_rel_acc
0,0,1,339,39,46811300,51.067703,10.614471,5.288548,2.192813,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,342,42,46811400,53.541672,10.643722,5.288548,2.192813,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1,345,45,46811500,55.993866,10.646164,5.288548,2.192813,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Pairing
* 차로를 막론하고, 모든 (타겟차량, 선행차량)의 페어링을 확보

In [17]:
vehicle_list = list(df['Vehicle ID'].unique()) # 모든 타겟차량 리스트

In [18]:
pairs = []

for veh in vehicle_list:
    veh_df = df[df['Vehicle ID'] == veh] # 각 차량
    
    LV0_ID_list = list(veh_df['LV0_ID'].unique())
    LVR_ID_list = list(veh_df['LVR_ID'].unique())
    LVL_ID_list = list(veh_df['LVL_ID'].unique())
    
    for LV0_ID in LV0_ID_list:    
        if pd.isna(LV0_ID) == False: # None 또는 np.NaN이 아니면 :: LV0이 있으면
            pairs.append((veh, int(LV0_ID), 'LV0'))
        else:
            pass
        
    for LVR_ID in LVR_ID_list:    
        if pd.isna(LVR_ID) == False: # None 또는 np.NaN이 아니면
            pairs.append((veh, int(LVR_ID), 'LVR'))
        else:
            pass
    
    for LVL_ID in LVL_ID_list:    
        if pd.isna(LVL_ID) == False: # None 또는 np.NaN이 아니면
            pairs.append((veh, int(LVL_ID), 'LVL'))
        else:
            pass

In [21]:
pairs[-100:-80]

[(1542, 1538, 'LVR'),
 (1542, 1543, 'LVR'),
 (1542, 1539, 'LVL'),
 (1543, 1542, 'LVL'),
 (1543, 1539, 'LVL'),
 (1544, 1542, 'LV0'),
 (1544, 1543, 'LVR'),
 (1544, 1542, 'LVL'),
 (1544, 1539, 'LVL'),
 (1545, 1539, 'LV0'),
 (1545, 1544, 'LVR'),
 (1545, 1542, 'LVR'),
 (1545, 1539, 'LVL'),
 (1546, 1545, 'LV0'),
 (1546, 1539, 'LV0'),
 (1546, 1544, 'LVR'),
 (1546, 1545, 'LVR'),
 (1547, 1546, 'LVL'),
 (1550, 1545, 'LV0'),
 (1550, 1547, 'LVR')]

# Longtype
* pairs 리스트의 튜플(TV, LV)에 따라서 각 차량의 위치, 속도, 가속도 등을 df로부터 프레임별로 뽑아오기

In [22]:
long = pd.DataFrame({
    # 타겟 차량
    'Vehicle ID': [], 'LV_ID': [], 'LV_type' : [], # LV_type : LV0, LVL, LVR 표시 
    'frame' : [], 'local_x' : [], 'local_y' : [], 'V_len' : [], 'V_wid' : [],
    'velocity' : [], 'acc' : [], 'Lane Identification' : [], 'Vehicle Movement' : [], 'Time' : [], 
    'Local X (m)_before' : [], 'Local Y(m)_before' : [], 'delta_local_x' : [], 'delta_local_y' : [],
    'velocity_x' : [], 'velocity_y' : [], 'velocity_x_before' : [], 'velocity_y_before' : [], 'delta_velocity_x' : [], 'delta_velocity_y' : [],
    'acc_x' : [], 'acc_y' : [], 'cell' : [],
    'Lane Identification Past' : [], 'Lane_record' : [], 'Lane_record_split' : [], 'Lane_00' : [], 'Lane_99' : [], 'Lane_change' : [],
    'Lane_leave' : [], 'Lane_change_direction' : [], 'LC_CF' : [],
    
    # 선행 차량
    
    'LV_local_x' : [], 'LV_local_y' : [], 'LV_len' : [], 'LV_wid' : [],
    'LV_velocity' : [], 'LV_acc' : [], 'LV_Lane Identification' : [], 'LV_Vehicle Movement' : [], 'LV_Time' : [], 
    'LV_Local X (m)_before' : [], 'LV_Local Y(m)_before' : [], 'LV_delta_local_x' : [], 'LV_delta_local_y' : [],
    'LV_velocity_x' : [], 'LV_velocity_y' : [], 'LV_velocity_x_before' : [], 'LV_velocity_y_before' : [], 
    'LV_delta_velocity_x' : [], 'LV_delta_velocity_y' : [],
    'LV_acc_x' : [], 'LV_acc_y' : [], 'LV_cell' : [],
    'LV_Lane Identification Past' : [], 'LV_Lane_record' : [], 'LV_Lane_record_split' : [], 'LV_Lane_00' : [], 'LV_Lane_99' : [], 'LV_Lane_change' : [],
    'LV_Lane_leave' : [], 'LV_Lane_change_direction' : [],    
    'LV_LC_CF' : []
})

In [23]:
for pair in tqdm(pairs):
    veh = pair[0]
    LV = pair[1]
    LV_type = pair[2]
    
    target_df = df[df['Vehicle ID'] == veh]
    LV_df = df[df['Vehicle ID'] == LV]
    
    target_frames = list(target_df['frame'].unique()) # 타겟차량의 등장프레임
    
    for frm in target_frames: # 각 타겟 차량의 등장 프레임별로 레코드를 집어넣어야 한다
        target_df_frm = target_df[target_df['frame'] == frm]
        LV_df_frm = LV_df[LV_df['frame'] == frm]
        
        if len(LV_df_frm) > 0: # 해당 프레임에 선행차량이 있을 경우에만 행 추가

            long_frm = pd.DataFrame({
                # 타겟 차량정보
                'Vehicle ID': [veh],
                'LV_ID' : [LV],
                'LV_type' : [LV_type], # LV_type : LV0, LVL, LVR 표시 
                'frame' : [frm],
                'local_x' : [target_df_frm['local_x'].iloc[0]],
                'local_y' : [target_df_frm['local_y'].iloc[0]],
                'V_len' : [target_df_frm['V_len'].iloc[0]], 
                'V_wid' : [target_df_frm['V_wid'].iloc[0]],
                'velocity' : [target_df_frm['velocity'].iloc[0]], 
                'acc' : [target_df_frm['acc'].iloc[0]], 
                'Lane Identification' : [target_df_frm['Lane Identification'].iloc[0]], 
                'Vehicle Movement' : [target_df_frm['Vehicle Movement'].iloc[0]], 
                'Time' : [target_df_frm['Time'].iloc[0]], 
                'Local X (m)_before' : [target_df_frm['Local X (m)_before'].iloc[0]], 
                'Local Y(m)_before' : [target_df_frm['Local Y(m)_before'].iloc[0]], 
                'delta_local_x' : [target_df_frm['delta_local_x'].iloc[0]], 
                'delta_local_y' : [target_df_frm['delta_local_y'].iloc[0]],
                'velocity_x' : [target_df_frm['velocity_x'].iloc[0]], 
                'velocity_y' : [target_df_frm['velocity_y'].iloc[0]], 
                'velocity_x_before' : [target_df_frm['velocity_x_before'].iloc[0]], 
                'velocity_y_before' : [target_df_frm['velocity_y_before'].iloc[0]], 
                'delta_velocity_x' : [target_df_frm['delta_velocity_x'].iloc[0]], 
                'delta_velocity_y' : [target_df_frm['delta_velocity_y'].iloc[0]],
                'acc_x' : [target_df_frm['acc_x'].iloc[0]], 
                'acc_y' : [target_df_frm['acc_y'].iloc[0]], 
                'cell' : [target_df_frm['cell'].iloc[0]],
                'Lane Identification Past' : [target_df_frm['Lane Identification Past'].iloc[0]], 
                'Lane_record' : [target_df_frm['Lane_record'].iloc[0]], 
                'Lane_record_split' : [target_df_frm['Lane_record_split'].iloc[0]], 
                'Lane_00' : [target_df_frm['Lane_00'].iloc[0]], 
                'Lane_99' : [target_df_frm['Lane_99'].iloc[0]], 
                'Lane_change' : [target_df_frm['Lane_change'].iloc[0]],
                'Lane_leave' : [target_df_frm['Lane_leave'].iloc[0]], 
                'Lane_change_direction' : [target_df_frm['Lane_change_direction'].iloc[0]],
                'LC_CF' : [target_df_frm['LC_CF'].iloc[0]],

                # 선행 차량정보
                'LV_local_x' : [LV_df_frm['local_x'].iloc[0]], 
                'LV_local_y' : [LV_df_frm['local_y'].iloc[0]], 
                'LV_len' : [LV_df_frm['V_len'].iloc[0]], 
                'LV_wid' : [LV_df_frm['V_wid'].iloc[0]],
                'LV_velocity' : [LV_df_frm['velocity'].iloc[0]], 
                'LV_acc' : [LV_df_frm['acc'].iloc[0]], 
                'LV_Lane Identification' : [LV_df_frm['Lane Identification'].iloc[0]], 
                'LV_Vehicle Movement' : [LV_df_frm['Vehicle Movement'].iloc[0]], 
                'LV_Time' : [LV_df_frm['Time'].iloc[0]], 
                'LV_Local X (m)_before' : [LV_df_frm['Local X (m)_before'].iloc[0]], 
                'LV_Local Y(m)_before' : [LV_df_frm['Local Y(m)_before'].iloc[0]], 
                'LV_delta_local_x' : [LV_df_frm['delta_local_x'].iloc[0]], 
                'LV_delta_local_y' : [LV_df_frm['delta_local_y'].iloc[0]],
                'LV_velocity_x' : [LV_df_frm['velocity_x'].iloc[0]], 
                'LV_velocity_y' : [LV_df_frm['velocity_y'].iloc[0]], 
                'LV_velocity_x_before' : [LV_df_frm['velocity_x_before'].iloc[0]], 
                'LV_velocity_y_before' : [LV_df_frm['velocity_y_before'].iloc[0]], 
                'LV_delta_velocity_x' : [LV_df_frm['delta_velocity_x'].iloc[0]], 
                'LV_delta_velocity_y' : [LV_df_frm['delta_velocity_y'].iloc[0]],
                'LV_acc_x' : [LV_df_frm['acc_x'].iloc[0]], 
                'LV_acc_y' : [LV_df_frm['acc_y'].iloc[0]], 
                'LV_cell' : [LV_df_frm['cell'].iloc[0]],
                'LV_Lane Identification Past' : [LV_df_frm['Lane Identification Past'].iloc[0]], 
                'LV_Lane_record' : [LV_df_frm['Lane_record'].iloc[0]], 
                'LV_Lane_record_split' : [LV_df_frm['Lane_record_split'].iloc[0]], 
                'LV_Lane_00' : [LV_df_frm['Lane_00'].iloc[0]], 
                'LV_Lane_99' : [LV_df_frm['Lane_99'].iloc[0]], 
                'LV_Lane_change' : [LV_df_frm['Lane_change'].iloc[0]],
                'LV_Lane_leave' : [LV_df_frm['Lane_leave'].iloc[0]], 
                'LV_Lane_change_direction' : [LV_df_frm['Lane_change_direction'].iloc[0]],
                'LV_LC_CF' : [LV_df_frm['LC_CF'].iloc[0]],
            })

            long = pd.concat([long, long_frm])
            
        else: # 해당 프레임에 선행차량이 없을 경우 패스해버림
            pass

# 모든 Iteration 돌고 난 뒤 long의 ID 열 타입 조정 : 정수화
long['Vehicle ID'] = long['Vehicle ID'].astype('int')
long['LV_ID'] = long['LV_ID'].astype('int')

# 거리관련 변수
long['D_x'] = long['LV_local_x'] - long['local_x']
long['D_y'] = long['LV_local_y'] - long['local_y']
long['D'] = (long['D_x']**2 + long['D_y']**2) ** (1/2)

100%|██████████████████████████████████████████████████████████████████████████████| 2188/2188 [45:14<00:00,  1.24s/it]


In [37]:
long.rename({'Lane Identification' : 'lane', 'LV_Lane Identification' : 'LV_lane'}, axis = 1, inplace = True)

In [38]:
long.columns

Index(['Vehicle ID', 'LV_ID', 'LV_type', 'frame', 'local_x', 'local_y',
       'V_len', 'V_wid', 'velocity', 'acc', 'lane', 'Vehicle Movement', 'Time',
       'Local X (m)_before', 'Local Y(m)_before', 'delta_local_x',
       'delta_local_y', 'velocity_x', 'velocity_y', 'velocity_x_before',
       'velocity_y_before', 'delta_velocity_x', 'delta_velocity_y', 'acc_x',
       'acc_y', 'cell', 'Lane Identification Past', 'Lane_record',
       'Lane_record_split', 'Lane_00', 'Lane_99', 'Lane_change', 'Lane_leave',
       'Lane_change_direction', 'LC_CF', 'LV_local_x', 'LV_local_y', 'LV_len',
       'LV_wid', 'LV_velocity', 'LV_acc', 'LV_lane', 'LV_Vehicle Movement',
       'LV_Time', 'LV_Local X (m)_before', 'LV_Local Y(m)_before',
       'LV_delta_local_x', 'LV_delta_local_y', 'LV_velocity_x',
       'LV_velocity_y', 'LV_velocity_x_before', 'LV_velocity_y_before',
       'LV_delta_velocity_x', 'LV_delta_velocity_y', 'LV_acc_x', 'LV_acc_y',
       'LV_cell', 'LV_Lane Identification Past', '

# Save

In [39]:
save_folder = '02_processed_long'
save_name = data_name
save_path = os.path.join(working_dir, save_folder, save_name)

In [40]:
long.to_csv(save_path, encoding = 'cp949')